<a href="https://colab.research.google.com/github/jideexploit3211/Davidbots/blob/Sensors/Sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install necessary packages
!pip install pybullet imageio-ffmpeg

import pybullet as p
import pybullet_data
import numpy as np
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML

# URDF description of a simple robot
robot_urdf = """
<robot name="simple_robot">
    <link name="torso">
        <visual>
            <geometry>
                <box size="0.4 0.4 0.4"/>
            </geometry>
            <material name="red">
                <color rgba="1 0 0 1"/>
            </material>
        </visual>
        <collision>
            <geometry>
                <box size="0.4 0.4 0.4"/>
            </geometry>
        </collision>
        <inertial>
            <mass value="10"/>
            <inertia ixx="0.1" iyy="0.1" izz="0.1"/>
        </inertial>
    </link>

    <link name="arm">
        <visual>
            <geometry>
                <box size="0.4 0.4 0.4"/>
            </geometry>
            <material name="green">
                <color rgba="0 1 0 1"/>
            </material>
        </visual>
        <collision>
            <geometry>
                <box size="0.4 0.4 0.4"/>
            </geometry>
        </collision>
        <inertial>
            <mass value="5"/>
            <inertia ixx="0.05" iyy="0.05" izz="0.05"/>
        </inertial>
    </link>

    <link name="leg">
        <visual>
            <geometry>
                <box size="0.4 0.4 0.4"/>
            </geometry>
            <material name="blue">
                <color rgba="0 0 1 1"/>
            </material>
        </visual>
        <collision>
            <geometry>
                <box size="0.4 0.4 0.4"/>
            </geometry>
        </collision>
        <inertial>
            <mass value="5"/>
            <inertia ixx="0.05" iyy="0.05" izz="0.05"/>
        </inertial>
    </link>

    <joint name="torso_to_arm" type="revolute">
        <parent link="torso"/>
        <child link="arm"/>
        <origin xyz="0 0.3 -0.37" rpy="0 0 0"/>
        <axis xyz="1 0 0"/>
        <limit lower="-1.57" upper="1.57"/>
    </joint>

    <joint name="torso_to_leg" type="revolute">
        <parent link="torso"/>
        <child link="leg"/>
        <origin xyz="0 -0.3 -0.37" rpy="0 0 0"/>
        <axis xyz="1 0 0"/>
        <limit lower="-1.57" upper="1.57"/>
    </joint>
</robot>
"""

# Write the URDF string to a file
with open('simple_robot.urdf', 'w') as f:
    f.write(robot_urdf)

# Connect to PyBullet in DIRECT mode
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.8)

# Load the plane and the robot
planeId = p.loadURDF("plane.urdf")
robotId = p.loadURDF("simple_robot.urdf", [0, 0, 1])

# Adjust the physics engine parameters to prevent sinking
p.changeDynamics(robotId, -1, lateralFriction=1.0, restitution=0.5)

# Camera parameters
cam_target_pos = [0, 0, 0]
cam_distance = 3
cam_yaw, cam_pitch, cam_roll = 50, -30, 0
cam_width, cam_height = 480, 368
cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 1], 2, 0.01, 100, 60

# Initialize video
vid = imageio_ffmpeg.write_frames('robot_simulation.mp4', (cam_width, cam_height), fps=30)
vid.send(None)

# Simulate the environment and capture frames
for t in range(240):  # Simulate for 240 steps (8 seconds at 30 fps)
    # Simulated sensor input (e.g., based on proximity or arbitrary function)
    sensor_input = np.sin(2 * np.pi * t / 120)

    # Move the leg based on sensor input
    p.setJointMotorControl2(robotId, 1, p.POSITION_CONTROL, targetPosition=0.5 * sensor_input)

    # Step the simulation
    p.stepSimulation()

    # Capture camera image
    cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
    cam_projection_matrix = p.computeProjectionMatrixFOV(cam_fov, cam_width / cam_height, cam_near_plane, cam_far_plane)
    image = p.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]

    # Add image to video
    vid.send(np.ascontiguousarray(image))

    # Update yaw for a rotating camera effect
    cam_yaw -= 0

# Close video
vid.close()
p.disconnect()

# Display video in notebook
mp4 = open('robot_simulation.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)